### 实例演示:
+ 1.将一个大excel等份拆分成多个excel
+ 2.将多个小的excel合并成一个大的excel并标记来源

In [1]:
work_dir = "./course_datas/c15_excel_split_merge/"
splits_dir = f"{work_dir}/splits"

import os
if not os.path.exists(splits_dir):
    os.mkdir(splits_dir)

## 0.读取源excel到pandas

In [2]:
import pandas as pd 

In [5]:
df_source = pd.read_excel(f"{work_dir}/crazyant_blog_articles_source.xlsx")

In [6]:
df_source.head()

,id,title,tags
0,2585,Tensorflow怎样接收变长列表特征,"python,tensorflow,特征工程"
1,2583,Pandas实现数据的合并concat,"pandas,python,数据分析"
2,2574,Pandas的Index索引有什么用途？,"pandas,python,数据分析"
3,2564,机器学习常用数据集大全,"python,机器学习"
4,2561,一个数据科学家的修炼路径,数据分析


In [7]:
df_source.index

RangeIndex(start=0, stop=258, step=1)

In [8]:
df_source.shape

(258, 3)

In [9]:
total_row_count = df_source.shape[0]

In [10]:
total_row_count

258

# 一.将一个大excel等份拆成多个excel
#### 1.使用df.iloc方法,将一个大的dataframe,拆分成多个小的dataframe
#### 2.将使用dataframe.to_excel保存每个小excel

## 1.计算拆分后的每个excel的行数

In [11]:
# 将这个大excel拆分给这几个人
user_names = ["xiao_shuai","xiao_huang","xiao_dou","xiao_ya","xiao_jiang","xiao_liu"]

In [13]:
# 每个人的任务数目
split_size = total_row_count // len(user_names)
if total_row_count % len(user_names) != 0:
    split_size += 1
    
split_size

43

## 2.拆分成多个dataframe

In [15]:
df_subs = []
for index,user_name in enumerate(user_names):
    # iloc的开始索引
    begin = index * split_size
    # iloc的结束索引
    end = begin + split_size 
    # 实现df按照iloc拆分
    df_sub  =df_source.iloc[begin:end]
    # 将每个子df存入列表
    df_subs.append((index,user_name,df_sub))
    

## 3.将每个dataframe存入excel

In [17]:
for index ,user_name,df_sub in df_subs:
    file_name = f"{splits_dir}/crazyant_blog_articles_{index}_{user_name}.xlsx"
    df_sub.to_excel(file_name,index=False)

# 二.合并多个小excel到一个大excel

##  1.遍历文件夹,得到要合并的excel名称列表

In [18]:
import os 
excel_names = []
for excel_name in os.listdir(splits_dir):
    excel_names.append(excel_name)
excel_names

['crazyant_blog_articles_0_xiao_shuai.xlsx',
 'crazyant_blog_articles_1_xiao_huang.xlsx',
 'crazyant_blog_articles_2_xiao_dou.xlsx',
 'crazyant_blog_articles_3_xiao_ya.xlsx',
 'crazyant_blog_articles_4_xiao_jiang.xlsx',
 'crazyant_blog_articles_5_xiao_liu.xlsx']

## 2.分别读取到dataframe

In [23]:
df_list = []

for excel_name in excel_names:
    # 读取每个excel到df
    excel_path = f"{splits_dir}/{excel_name}"
    df_split = pd.read_excel(excel_path)
    # 得到username
    username = excel_name.replace("crazyant_blog_articles_","").replace(".xlsx","")[2:]
    print(username)
    # 给每个df添加一列,即用户名字
    df_split["username"] = username
    df_list.append(df_split)

xiao_shuai
xiao_huang
xiao_dou
xiao_ya
xiao_jiang
xiao_liu


## 3.使用pd.concat进行合并

In [24]:
df_merged = pd.concat(df_list)

In [25]:
df_merged.shape

(258, 4)

In [26]:
df_merged.head()

,id,title,tags,username
0,2585,Tensorflow怎样接收变长列表特征,"python,tensorflow,特征工程",xiao_shuai
1,2583,Pandas实现数据的合并concat,"pandas,python,数据分析",xiao_shuai
2,2574,Pandas的Index索引有什么用途？,"pandas,python,数据分析",xiao_shuai
3,2564,机器学习常用数据集大全,"python,机器学习",xiao_shuai
4,2561,一个数据科学家的修炼路径,数据分析,xiao_shuai


In [27]:
df_merged["username"].value_counts()

xiao_jiang    43
xiao_liu      43
xiao_ya       43
xiao_shuai    43
xiao_dou      43
xiao_huang    43
Name: username, dtype: int64

##  4.将合并后的dataframe输出到excel

In [28]:
df_merged.to_excel(f"{work_dir}/blog_merged.xlsx",index=False)